In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append("../")
import os
import json

import logging
from src.utils import logging_utils
from src.utils import env_utils

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(
    f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}"
)
logger.info(f"{transformers.__version__=}")

/share/u/arnab/miniconda3/envs/connection/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-04-14 17:20:57 __main__ INFO     torch.__version__='2.6.0+cu124', torch.version.cuda='12.4'
2025-04-14 17:20:57 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=1, torch.cuda.get_device_name()='NVIDIA RTX A6000'
2025-04-14 17:20:57 __main__ INFO     transformers.__version__='4.51.2'


In [3]:
os.listdir(os.path.join(env_utils.DEFAULT_MODELS_DIR, "meta-llama"))

['Llama-3.1-8B-Instruct', 'Llama-3.1-8B']

In [4]:
import torch

from src.models import ModelandTokenizer

# model_key = "meta-llama/Llama-3.1-8B"
# model_key = "meta-llama/Llama-3.2-3B"
# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-2-27b-it"
# model_key = "Qwen/Qwen2-7B"
# model_key = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
# model_key = "allenai/OLMo-2-1124-7B-Instruct"
# model_key = "allenai/OLMo-7B-0424-hf"

model_key = "Qwen/Qwen2.5-14B"

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.float16,
)

2025-04-14 17:21:06 src.models WARNING  Qwen/Qwen2.5-14B not found in /share/u/models
If not found in cache, model will be downloaded from HuggingFace to cache directory


2025-04-14 17:21:06 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2025-04-14 17:21:08 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /Qwen/Qwen2.5-14B/resolve/main/config.json HTTP/1.1" 200 0
2025-04-14 17:21:08 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /Qwen/Qwen2.5-14B/resolve/main/tokenizer_config.json HTTP/1.1" 200 0


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


2025-04-14 17:21:10 accelerate.utils.modeling INFO     We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards: 100%|██████████| 8/8 [00:04<00:00,  1.80it/s]


2025-04-14 17:21:15 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /Qwen/Qwen2.5-14B/resolve/main/generation_config.json HTTP/1.1" 200 0
2025-04-14 17:21:15 src.models INFO     loaded model <Qwen/Qwen2.5-14B> | size: 28171.604 MB | dtype: torch.float16 | device: cuda:0


In [8]:
from src.probing.utils import (
    prepare_probing_input,
    get_lm_generated_answer,
    check_if_answer_is_correct,
)

Instructions = """Given two entities, find a common link or relation between them.
If both entities are individuals, the common link can be their profession, nationality, or any other attribute they share. Their relation can be if someone is the student/teacher of the other etc.
Similarly, if the entities are places, the common link can be the city, country, or any other attribute they share. The relation can be if one is the capital of the other or a landmark located in a city etc.
If there is no connection just answer "None"."""

# Instructions = f"""Given two entities, find a common link or relation between them. If there is no connection just answer "None"."""

block_separator = "\n#"
question_marker = "\nQ: "
answer_marker = "\nA:"

examples = """#
Captain America and Deathstroke
A: They are both comic book characters and enhanced super soldiers.
#
Q: Tiger Woods and Phil Mickelson
A: They are both professional golfers.
#
Q: Rome and Italy
A: Rome is the capital city of Italy.
#
Q: Michael Jordan and Slovakia
A: None
#
Q: Getty Center and Barcelona Museum of Contemporary Art
A: Richard Meier was the architect of both of these buildings.
"""

with open(os.path.join(env_utils.DEFAULT_DATA_DIR, "coincidences_sample.json")) as f:
    coincidences = json.load(f)

logger.info(f"{len(coincidences['examples'])=}")


# entities = coincidences["examples"][0]["entity_pair"]
# entities = ("Whale", "Dolphin")
# entities = ("Nautilus", "Dolphin")
# entities = ("Abraham Lincoln", "John F. Kennedy")
# entities = ("Brad Pitt", "Angelina Jolie")
# entities = ("Emu", "Ostrich")
# entities = ("Elephant", "Whale")
# entities = ("Wolverine", "Penguin")
# entities = ("Giraffe", "Reindeer")
# entities = ("Hydrogen", "Oxygen")
# entities = ("Alfred Nobel", "Julius Caesar")
# entities = ("Cleopatra", "Catherine the Great")
# entities = ("Snake", "Spider")
# entities = ("Tomato", "Banana")
# entities = ("Diamond", "Coal")
# entities = ("Rome", "Istanbul")
# entities = ("Mercury", "Jupiter")
# entities = ("butterflies", "frogs")
# entities = ("octopus", "squid")
# entities = ("Kangaroo", "Seahorse")
# entities = ("Great Wall of China", "Panama Canal")
# entities = ("Brazil", "Turkey")
# entities = ("onion", "garlic")
# entities = ("jellyfish", "lobster")
# entities = ("corn", "wheat")
# entities = ("broccoli", "cauliflower")
# entities = ("Venus", "Uranus")
# entities = ("crocodile", "shark")
# entities = ("crab", "spider")
# entities = ("apple", "rose")
# entities = ("starfish", "lizard")
# entities = ("zebra", "penguin")
# entities = ("rabbit", "deer")
# entities = ("mushroom", "coral")
# entities = ("pinaapple", "fig")
# entities = ("pig", "dolphin")
# entities = ("cinnamon", "vanilla")
# entities = ("tuna", "hummingbird")
# entities = ("diamond", "graphaite")
# entities = ("copper", "gold")

# entities = ("apple", "windows")
# entities = ("doors", "windows")

entities = ("Tom Cruise", "Brad Pitt")

print(entities)

prefix = f"""{Instructions}
{examples}
"""

prompt = prepare_probing_input(
    mt=mt,
    entities=entities,
    prefix=prefix,
    answer_marker=answer_marker,
    question_marker=question_marker,
    block_separator=block_separator,
    is_a_reasoning_model="deepseek" in model_key.lower(),
    # is_a_reasoning_model=True
    answer_prefix=" They are/were both",
    # answer_prefix = " They are both used to say"
)

print(mt.tokenizer.decode(prompt.tokenized["input_ids"][0]))

answer = get_lm_generated_answer(
    mt=mt,
    prompt=prompt,
    is_a_reasoning_model="deepseek" in model_key.lower(),
    # is_a_reasoning_model=True
)
print(f"{answer=}")

2025-04-14 17:27:15 __main__ INFO     len(coincidences['examples'])=13


('Tom Cruise', 'Brad Pitt')
Given two entities, find a common link or relation between them.
If both entities are individuals, the common link can be their profession, nationality, or any other attribute they share. Their relation can be if someone is the student/teacher of the other etc.
Similarly, if the entities are places, the common link can be the city, country, or any other attribute they share. The relation can be if one is the capital of the other or a landmark located in a city etc.
If there is no connection just answer "None".
#
Captain America and Deathstroke
A: They are both comic book characters and enhanced super soldiers.
#
Q: Tiger Woods and Phil Mickelson
A: They are both professional golfers.
#
Q: Rome and Italy
A: Rome is the capital city of Italy.
#
Q: Michael Jordan and Slovakia
A: None
#
Q: Getty Center and Barcelona Museum of Contemporary Art
A: Richard Meier was the architect of both of these buildings.
#
Q: Tom Cruise and Brad Pitt
A: They are/were both
answer

In [6]:

check_if_answer_is_correct(
    answer=answer,
    entities=entities,
)

2025-04-14 17:21:48 anthropic._base_client DEBUG    Request options: {'method': 'post', 'url': '/v1/messages', 'timeout': Timeout(connect=5.0, read=600, write=600, pool=600), 'files': None, 'json_data': {'max_tokens': 4000, 'messages': [{'role': 'user', 'content': [{'type': 'text', 'text': 'Do you think the following answer is a good connection or relation between the entities doors and windows?\nYour answer should start with "Yes" or "No". If the answer is "No", please provide your reasoning. Otherwise, just say "Yes".\n\n\nused to enter and exit a building.'}]}], 'model': 'claude-3-5-sonnet-20241022', 'system': 'You are a helpful assistant.', 'temperature': 0}}
2025-04-14 17:21:48 anthropic._base_client DEBUG    Sending HTTP Request: POST https://api.anthropic.com/v1/messages
2025-04-14 17:21:48 httpcore.connection DEBUG    connect_tcp.started host='api.anthropic.com' port=443 local_address=None timeout=5.0 socket_options=[(1, 9, True), (6, 5, 60), (6, 6, 5), (6, 4, 60)]
2025-04-14 1

False

In [9]:
from src.functional import generate_with_patch, predict_next_token
from src.utils.typing import TokenizerOutput

print(entities)

clean_pred = predict_next_token(
    mt=mt,
    inputs=TokenizerOutput(data=prompt.tokenized),
)
clean_pred

('Tom Cruise', 'Brad Pitt')
2025-04-14 17:27:37 httpcore.connection DEBUG    close.started
2025-04-14 17:27:37 httpcore.connection DEBUG    close.complete


[[PredictedToken(token=' actors', prob=0.36083984375, logit=19.25, token_id=19571),
  PredictedToken(token=' married', prob=0.1900634765625, logit=18.609375, token_id=12224),
  PredictedToken(token=' Hollywood', prob=0.06671142578125, logit=17.5625, token_id=17236),
  PredictedToken(token=' famous', prob=0.06365966796875, logit=17.515625, token_id=11245),
  PredictedToken(token=' movie', prob=0.03863525390625, logit=17.015625, token_id=5700)]]

In [10]:
interesting_tokens = [19571]
# interesting_tokens = []

In [11]:
single_probing_prompt = "Who/what is a {}? Answer: {} is a"

gen = generate_with_patch(
    mt=mt,
    inputs=single_probing_prompt.format(entities[0], entities[0]),
)

print(gen)

pred, track_ans = predict_next_token(
    mt=mt,
    inputs=[single_probing_prompt.format(entities[0], entities[0])],
    k=15,
    token_of_interest=[clean_pred[0][0].token_id] + interesting_tokens,
)

print(track_ans)
pred

['Who/what is a Tom Cruise? Answer: Tom Cruise is a world-renowned actor who has been in the entertainment industry for almost 4 decades now. He is', 'Who/what is a Tom Cruise? Answer: Tom Cruise is a very famous actor. He stars in lots of popular movies. He has lots of fans all over the', 'Who/what is a Tom Cruise? Answer: Tom Cruise is a well-known American actor who has starred in many blockbuster movies. Born on July 3, 1', 'Who/what is a Tom Cruise? Answer: Tom Cruise is a talented and accomplished American actor, director, producer, and motivational speaker who has made a significant impact on', 'Who/what is a Tom Cruise? Answer: Tom Cruise is a highly popular actor, who stars in block-buster movies.']
[{19571: (3517, PredictedToken(token=' actors', prob=1.6689300537109375e-06, logit=5.9453125, token_id=19571))}]


[[PredictedToken(token=' famous', prob=0.375732421875, logit=18.265625, token_id=11245),
  PredictedToken(token=' very', prob=0.08001708984375, logit=16.71875, token_id=1602),
  PredictedToken(token=' well', prob=0.07171630859375, logit=16.609375, token_id=1632),
  PredictedToken(token=' movie', prob=0.05328369140625, logit=16.3125, token_id=5700),
  PredictedToken(token=' popular', prob=0.047027587890625, logit=16.1875, token_id=5411),
  PredictedToken(token=' person', prob=0.0298919677734375, logit=15.734375, token_id=1697),
  PredictedToken(token=' ', prob=0.0210418701171875, logit=15.3828125, token_id=220),
  PredictedToken(token=' successful', prob=0.018707275390625, logit=15.265625, token_id=6849),
  PredictedToken(token=' man', prob=0.0185699462890625, logit=15.2578125, token_id=883),
  PredictedToken(token=' talented', prob=0.01491546630859375, logit=15.0390625, token_id=23074),
  PredictedToken(token=' Hollywood', prob=0.01468658447265625, logit=15.0234375, token_id=17236),
  

In [170]:
gen = generate_with_patch(
    mt=mt,
    inputs=single_probing_prompt.format(entities[1], entities[1]),
)

print(gen)

pred, track_ans = predict_next_token(
    mt=mt,
    inputs=single_probing_prompt.format(entities[1], entities[1]),
    k=15,
    token_of_interest=[clean_pred[0][0].token_id] + interesting_tokens,
)

print(track_ans)
pred

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['Who/what is a gold? Answer: gold is a chemical element with the symbol Au (from Latin: aurum) and an atomic number of 79', 'Who/what is a gold? Answer: gold is a precious metal that is used as a currency and is traded on the stock market. Gold is also used', 'Who/what is a gold? Answer: gold is a person who is 18 years of age or older and is registered with the College of Physicians and Surge', 'Who/what is a gold? Answer: gold is a metallic element, chemical symbol Au, atomic number 79, atomic weight 196.967, density', 'Who/what is a gold? Answer: gold is a metal that is yellow in color. It is malleable, meaning it can be beaten into thin']
[{11667: (30, PredictedToken(token=' currency', prob=0.004764556884765625, logit=9.8359375, token_id=11667)), 1511: (1868, PredictedToken(token=' used', prob=3.528594970703125e-05, logit=4.9296875, token_id=1511))}]


[[PredictedToken(token=' chemical', prob=0.073974609375, logit=12.578125, token_id=11742),
  PredictedToken(token=' metal', prob=0.072265625, logit=12.5546875, token_id=9501),
  PredictedToken(token=' precious', prob=0.052032470703125, logit=12.2265625, token_id=27498),
  PredictedToken(token=' yellow', prob=0.04022216796875, logit=11.96875, token_id=14071),
  PredictedToken(token=' mineral', prob=0.036041259765625, logit=11.859375, token_id=25107),
  PredictedToken(token=' type', prob=0.0230865478515625, logit=11.4140625, token_id=955),
  PredictedToken(token=' person', prob=0.0184173583984375, logit=11.1875, token_id=1732),
  PredictedToken(token=' gold', prob=0.015869140625, logit=11.0390625, token_id=6761),
  PredictedToken(token=' symbol', prob=0.0144500732421875, logit=10.9453125, token_id=7891),
  PredictedToken(token=' rare', prob=0.01357269287109375, logit=10.8828125, token_id=9024),
  PredictedToken(token=' metallic', prob=0.01336669921875, logit=10.8671875, token_id=46258),


In [171]:
with open(os.path.join(env_utils.DEFAULT_DATA_DIR, "coincidences_sample.json")) as f:
    coincidences = json.load(f)

logger.info(f"{len(coincidences['examples'])=}")

2025-02-18 15:01:13 __main__ INFO     len(coincidences['examples'])=20
